In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter, PillowWriter
from matplotlib import cm
from matplotlib import colors
from mpl_toolkits.mplot3d.art3d import Line3DCollection

# --- Geometry & mode ---
ratio = 10/4
Lz = 1.0
Lx = ratio * Lz
m, n, c = 1, 1, 1.0
amplitude = 0.4  # Controls the max displacement

fps = 20
nx, nz = 160, 400
x = np.linspace(0.0, Lx, nx)
z = np.linspace(0.0, Lz, nz)
X, Z = np.meshgrid(x, z, indexing="xy")

Psi = np.sin(m*np.pi*X/Lx) * np.sin(n*np.pi*Z/Lz)
f_mn = 0.5 * c * np.sqrt((m*Lx)**-2 + (n/Lz)**-2)
T = 1.0 / f_mn

### MODIFIED ###
# Calculate frames for two full periods (2 * T) to double the length
nframes = int(round(2 * T * fps))
t_step = 1.0/fps

# --- Figure ---
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(111, projection='3d')
fig.patch.set_facecolor("white")
fig.subplots_adjust(0, 0, 1, 1)
ax.set_position([0, 0, 1, 1])
ax.set_axis_off()

# --- Colormap setup ---
norm = plt.Normalize(vmin=-2*amplitude, vmax=2*amplitude)

def tint_to_hex(cmap, hex_color="#a4bec4", saturation_factor=1.0):
    N = 256
    vals = cmap(np.linspace(0, 1, N))
    hsv = colors.rgb_to_hsv(vals[:, :3])
    target_hue = colors.rgb_to_hsv(np.array(colors.to_rgb(hex_color))[None, :])[0, 0]
    hsv[:, 0] = target_hue
    hsv[:, 1] *= saturation_factor
    vals[:, :3] = colors.hsv_to_rgb(hsv)
    return colors.LinearSegmentedColormap.from_list("hex_tint", vals)

cmap_base = plt.get_cmap("copper")
cmap = tint_to_hex(cmap_base, "#a4bec4", saturation_factor=0.5)

rng = np.random.default_rng(0)
dither = (rng.random(Psi.shape) - 0.5) * 1.0/255.0

def colorize(U, use_dither=False):
    V = U + (dither if use_dither else 0.0)
    V = np.clip(V, -amplitude, amplitude)
    return cmap(norm(V))

# --- Initial Surface Plot ---
U0 = amplitude * Psi * np.cos(0.0)
Xp, Yp, Zp = U0, Z, -X
surf = ax.plot_surface(
    Xp, Yp, Zp,
    facecolors=colorize(U0, use_dither=False),
    rcount=Z.shape[0], ccount=Z.shape[1],
    linewidth=0, edgecolor='none', antialiased=False, shade=False
)

# ### START: GRID DEFINITION ###

# --- Grid Parameters ---
grid_nx = 30
grid_nz = 15
grid_color = 'k'
grid_lw = 1.5
grid_eps = 1e-3 * max(Lx, Lz)    # constant normal-offset; tweak 5e-4..5e-3

dx = x[1] - x[0]
dz = z[1] - z[0]

def _grid_segments(U):
    # surface
    Xp, Yp, Zp = U, Z, -X

    # --- normal-based offset (prevents coplanarity even when U≈0)
    dUdx = np.gradient(U, dx, axis=1)
    dUdz = np.gradient(U, dz, axis=0)
    nx_ = 1.0
    ny_ = -dUdz
    nz_ = dUdx
    nrm = np.sqrt(nx_**2 + ny_**2 + nz_**2)
    nxh, nyh, nzh = nx_/nrm, ny_/nrm, nz_/nrm

    Xg = Xp + grid_eps * nxh
    Yg = Yp + grid_eps * nyh
    Zg = Zp + grid_eps * nzh

    x_indices = np.linspace(0, nx - 1, grid_nx, dtype=int)
    z_indices = np.linspace(0, nz - 1, grid_nz, dtype=int)

    segments_x, segments_z = [], []
    for j in z_indices:
        pts = np.array([Xg[j, :], Yg[j, :], Zg[j, :]]).T.reshape(-1, 1, 3)
        segments_x.extend(np.concatenate([pts[:-1], pts[1:]], axis=1))
    for i in x_indices:
        pts = np.array([Xg[:, i], Yg[:, i], Zg[:, i]]).T.reshape(-1, 1, 3)
        segments_z.extend(np.concatenate([pts[:-1], pts[1:]], axis=1))
    return segments_x, segments_z

# --- Initialize Grid Collections ---
segments_x_initial, segments_z_initial = _grid_segments(U0)
grid_x = Line3DCollection(segments_x_initial, colors=grid_color, linewidths=grid_lw)
grid_z = Line3DCollection(segments_z_initial, colors=grid_color, linewidths=grid_lw)

# draw grid last irrespective of mean depth
grid_x.set_sort_zpos(np.inf)
grid_z.set_sort_zpos(np.inf)

ax.add_collection3d(grid_x)
ax.add_collection3d(grid_z)
# ### END: GRID DEFINITION ###


# --- Axes and View Setup ---
### MODIFIED ###: Use grid_eps, which is defined.
max_extent = amplitude + grid_eps
ax.set_xlim(-max_extent, max_extent)
ax.set_ylim(0.0, Lz)
ax.set_zlim(-Lx, 0.0)
ax.view_init(elev=15, azim=10)

try:
    ax.set_box_aspect((2.0 * max_extent, Lz, Lx), zoom=1)
except TypeError:
    ax.set_box_aspect((2.0 * max_extent, Lz, Lx))
    ax.dist = 6

# --- Animation Update Function ---
def update(k, use_dither=False):
    global surf
    phase = np.cos(2 * np.pi * f_mn * (k * t_step))
    U = amplitude * Psi * phase
    Xp, Yp, Zp = U, Z, -X

    surf.remove()
    surf = ax.plot_surface(
        Xp, Yp, Zp,
        facecolors=colorize(U, use_dither=use_dither),
        rcount=Z.shape[0], ccount=Z.shape[1],
        linewidth=0, edgecolor='none', antialiased=False, shade=False
    )

    segx, segz = _grid_segments(U)
    grid_x.set_segments(segx)
    grid_z.set_segments(segz)

    return [surf, grid_x, grid_z]

# --- Generate Preview and Animation ---

print("Generating preview image...")
# Update to a frame where the membrane is nearly flat
update(nframes // 8 - 2)
plt.savefig("preview_with_grid_nearly_flat.png", dpi=400, pad_inches=0)
print("Saved: preview_with_grid_nearly_flat.png")


# ---------- Create Animation ----------

print("Starting animation rendering...")

### MODIFIED ###: Uncommented this section to save the video
# ---------- MP4 ----------
mp4_path = f"_mode({m},{n})_grid.mp4"
anim = FuncAnimation(fig, update, fargs=(False,), frames=nframes // 2,
                     interval=1000/fps, blit=False)
ffmpeg_writer = FFMpegWriter(fps=fps, codec="h264", extra_args=['-g', '1', '-pix_fmt', 'yuv420p'])
anim.save(mp4_path, writer=ffmpeg_writer,
          dpi=200, savefig_kwargs={'pad_inches': 0})
print("Saved:", mp4_path)

plt.close(fig)
print("Done.")

Generating preview image...
Saved: preview_with_grid_nearly_flat.png
Starting animation rendering...
Saved: _mode(1,1)_grid.mp4
Done.
